# @VerbenAufgabenBot: задания с пропусками для тренировки форм немецких глаголов

#pip install newspaper
#pip install spacy
#pip install pyTelegramBotAPI

In [1]:
import newspaper
from newspaper import Article
import spacy
import re
from collections import Counter
import telebot
from telebot import types
import unittest
import functools
import time

In [2]:
def timer(func):
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        runtime = time.perf_counter() - start
        print(f"{func.__name__} took {runtime:.4f} secs")
        return result
    return _wrapper

In [3]:
class getArticle:
    
    __article: Article
    __text: str
        
    def __init__(self) -> None:
        
        self.__article = Article 
        self.__text = ''
        
    def getText(self, url) -> str:
        """ На вход подаётся ссылка, на выход - текст статьи.
        """
        article = self.__article(url)
        article.download()
        article.parse()
        self.__text += article.text
        
        return self.__text
    
    def saveArticle(self):
        """ Сохраняет исходную статью в файл """
        file = open("text2.txt", 'w', encoding="utf-8")
        file.write(self.__text)
        file.close()

In [4]:
art = getArticle()

In [5]:
#a = art.getText("https://www.ferienwiki.de/feiertage/neujahr")
#print(a)

In [6]:
class tagWords(getArticle):
    
    __nlp: spacy.load
    __doc: spacy.tokens.doc.Doc
        
    def pos_text(self, text) -> str:
        
        self.__nlp = spacy.load('de_core_news_sm')
        self.__doc = (self.__nlp(text))
        
        pos_text = (' '.join('{word}/{tag}'.format(word=t.orth_, tag=t.pos_) for t in self.__doc))
        return(pos_text)

In [7]:
tag = tagWords()

In [8]:
#t = tag.pos_text(a)
#print(t)

In [9]:
class cleanVerbs:
    __clean_verb: list
    __clean_aux: list
    __clean_list: list
    
    def __init__(self) -> None:
        self.__clean_verb = []
        self.__clean_aux = []
        self.__clean_list = []
    
    def clean_list(self, text) -> list:
        
        verb_list = re.findall('[a-z,A-Z,äöü,ÄÖÜ]+/VERB', text)
        for i in verb_list:
            i = i[:-5]
            self.__clean_verb.append(i)
        
        aux_list = re.findall('[a-z,A-Z,äöü,ÄÖÜ]+/AUX', text)
        for i in aux_list:
            i = i[:-4]
            self.__clean_aux.append(i)
            
        self.__clean_list = self.__clean_verb + self.__clean_aux
        return(self.__clean_list)

In [10]:
c = cleanVerbs()

In [11]:
#C = c.clean_list(t)

In [12]:
class makeTask:
    l_x: list
    task: list
    
    def __init__(self) -> None:
        l_x = [] 
        task = []
    
    def task_text(self, text, verbs) -> list:
        l_x = text[0:-1].replace('"',"").split(' ')
        task = ['_____' if x in verbs else x for x in l_x]
        return(task)


In [13]:
#task = makeTask()
#t = task.task_text(a, C)
#print(t)

In [14]:
class findVerbs:
    __nlp: spacy.load
    raw_verbs: spacy.tokens.doc.Doc

    def __init__(self) -> None:
        self.__nlp = spacy.load('de_core_news_sm') 
    
    def lemmatize (self, text) -> list:
        raw_verbs = self.__nlp(' '.join(text))
        lemma_list = []
        for token in raw_verbs:
            lemma_list.append(token.lemma_)
        lemmas_quantity = str(dict(Counter(lemma_list)))
        return(lemmas_quantity)

In [15]:
#find = findVerbs()

#f = find.lemmatize(C)
#print(f)

In [16]:
#type(f)

In [17]:
@timer
class Facade:
    
    def __init__(self) -> None:
        self.get = getArticle()
        self.tag = tagWords()
        self.clean = cleanVerbs()
        self.verbs = findVerbs()
        self.task = makeTask()
        
        
    def Aufgabe(self, url) -> str:
        
        s = self.clean.clean_list(self.tag.pos_text(self.get.getText(url))) #вытаскиваем из текста все глаголы без тегов
        b = self.verbs.lemmatize(s) #приводим глаголы к начальной форме, указываем количество повторений
        c = (' '.join(self.task.task_text(self.get.getText(url), b))) #удаляем глаголы из текста статьи
        d = c + b
        d = ''.join(d)
        return (d)
    

In [18]:
make = Facade()

Facade took 2.8739 secs


In [19]:
#make.Aufgabe('https://www.ferienwiki.de/feiertage/heilige-drei-koenige')

In [20]:
bot = telebot.TeleBot('5763039812:AAF2Kh0sIk5dRT9giAt7rRSDWK8bk8puh28')

In [21]:
@bot.message_handler(commands=['start'])
def start(message):
    mess = f'Привет, {message.from_user.first_name}. Пришли мне ссылку на сайт на немецком языке, из которого ты хочешь сделать задание'
    bot.send_message(message.chat.id, mess, parse_mode='html')

In [22]:
@bot.message_handler()
def get_user_text(message):
    if message.text[0:4] == 'http':
        bot.register_next_step_handler(message, make.Aufgabe)
        
        f = open('new.txt', 'w', encoding="utf-8") #создаём файл и записываем в него текст с пропусками и глаголы
        f.write(make.Aufgabe(''.join(message.text)))
        f.close()
        with open('new.txt', "rb") as file:
            bot.send_document(message.chat.id, document=file)
                                             
    else:
        bot.send_message(message.chat.id, """ 
              Кажется, это не ссылка. Проверь формат.""")

In [ ]:
bot.polling(none_stop=True)